In [40]:
############################################################
# GeoHack 2023 Automated Fire Perimiter
#
#Provides functions to:
# 1.1 > Classify Raster
# 1.2 > Raster to Vector
#     > ??? Project Vectors ???
#     > Aggregate Polygons
#     > Split
#     > Combine
#
#Information:
#     Automated Fire Perimeter model using classification and
#     transformation techniques to rapidly build a fire
#     shape file. This model reduces noise and accounts for
#     layers of human subjectivity based on industry defined
#     threshold. Prectical applications for fire inteligence
#     are situational awareness of fire growth in a near 
#     real time environment. Each succesive update offers 
#     a POI for AI fire pretiction models from other platforms
# 
# Acknowledgments: GeoHack 2023 Team 1 ("Arno Roeder (Germany); Ricardo Lopes (Portugal); Sara Houser (Switzerland); 
#                  Flavia Ackermann (Switzerland); Curtis Doty (United States); ChatGPT (Open AI)
#          > Info: Special thanks to the team for supporting contibutions which were instrumental to the build.
#   
# By: Tony Ramos
# Date: 03/10/2023
#
############################################################

In [41]:
import os
import rasterio
import shapely
import numpy as np
import geopandas
import scipy

from matplotlib import pyplot




In [42]:
# File Path

work_dir = "./data/"
image = os.path.join(work_dir, "LWIR_QuickMosaic_16-bit_9327.tiff")
poly = os.path.join(work_dir, "HeatPoly.shp")
# work_dir ="Data"
# MillsFire="C:/Users/centr/GeoHack23-WildfirePerimeter/Data/LWIR_QuickMosaic_16-bit_9327.tiff"
OutputFilePath ="./data/"

In [43]:
# 1.1
# CLASSIFY RASTER

# Here's a step-by-step breakdown of the code:

# The code defines a threshold value KneeThresh which is used to classify the raster data later on.

# The with statement opens the input raster file using rasterio, and the raster data and metadata are 
# read and stored in raster_data and raster_meta variables, respectively.

# The numpy library is used to classify the raster data based on the threshold value. The np.where() 
# function is used to create a new array where all values less than KneeThresh are set to 0, and all 
# other values are set to 1. The resulting array is stored in the classified_data variable.

# The metadata of the input raster is updated to reflect the new data type and nodata value. 
# The rasterio.uint8 data type is used and nodata is set to None.

# The output raster file is created using the rasterio.open() function. The output file is named 
# RasterClass.tif and is located in the OutputFilePath directory. The metadata of the input raster 
# is passed to the function as **raster_meta which unpacks the dictionary into keyword arguments. 
# The w mode is used to open the file for writing.

# The dst.write() function writes the classified data to the output raster file. The astype() method 
# is used to convert the data type of the classified_data array to rasterio.uint16, which matches the 
# data type of the output raster file. The 1 parameter indicates that the data should be written to 
# the first band of the raster file.


In [44]:
#1.1
# Open the raster file using rasterio
KneeThresh = 33332
with rasterio.open(image) as src:
    # Read the raster data and metadata
    raster_data = src.read(1)  # Read the first band of the raster
    raster_meta = src.meta

    # Classify the raster data
    classified_data = np.where(raster_data < KneeThresh, 0, 1)

# Update the metadata with the new data type and nodata value
raster_meta.update(dtype=rasterio.uint8, nodata=None)

# Write the classified raster to a new file
with rasterio.open(OutputFilePath + 'RasterClass.tif', 'w', **raster_meta) as dst:
    dst.write(classified_data.astype(rasterio.uint16), 1)


In [45]:
# 1.2
# RASTER TO VECTOR

# The with statement opens the input raster file using rasterio, and the raster data and metadata are read 
# and stored in raster_data and raster_meta variables, respectively.

# The numpy library is used to classify the raster data based on a threshold value. The np.where() function 
# is used to create a new array where all values less than KneeThresh are set to 0, and all other values are set to 1. 
# The resulting array is stored in the classified_data variable.

# The metadata of the input raster is updated to reflect the new data type and nodata value. The rasterio.int16 
# data type is used and nodata is set to None.

# The output raster file is created using the rasterio.open() function. The output file is named RasterClass.tif 
# and is located in the OutputFilePath directory. The metadata of the input raster is passed to the function as 
# **raster_meta which unpacks the dictionary into keyword arguments. The w mode is used to open the file for writing.

# The dst.write() function writes the classified data to the output raster file. The 1 parameter indicates that the 
# data should be written to the first band of the raster file.

# The features.shapes() function from the rasterio.features module is used to convert the classified raster to polygons. 
# The transform parameter is used to provide the affine transformation matrix that maps the raster coordinates to real-world 
# coordinates.

# The fiona library is used to write the polygons to a shapefile. The output shapefile is named HeatPoly.shp 
# and is located in the OutputFilePath directory. The w mode is used to open the file for writing, and the 
# crs parameter is used to set the coordinate reference system (CRS) of the shapefile. The schema parameter is 
# used to define the schema of the shapefile, which in this case only includes a geometry field and no attribute fields.

# A loop is used to iterate over each polygon in the shapes list. The loop checks if the polygon has a nonzero value, 
# indicating that it belongs to a class, and creates a new feature dictionary with the polygon geometry and no attribute 
# values. The dst.write() function writes the feature to the output shapefile.


In [46]:
#1.2
import rasterio
from rasterio import features
import fiona
import matplotlib.pyplot as plt

# Open the raster file using rasterio
with rasterio.open(image) as src:
    # Read the raster data and metadata
    raster_data = src.read(1)  # Read the first band of the raster
    raster_meta = src.meta

    # Classify the raster data
    classified_data = np.where(raster_data < KneeThresh, 0, 1)

# Update the metadata with the new data type and nodata value
raster_meta.update(dtype=rasterio.int16, nodata=None)

# Write the classified raster to a new file
with rasterio.open(OutputFilePath + 'RasterClass.tif', 'w', **raster_meta) as dst:
    dst.write(classified_data, 1)

# Convert the classified raster to polygons
shapes = features.shapes(classified_data, transform=raster_meta['transform'])

# Write the polygons to a shapefile
with fiona.open(OutputFilePath + 'HeatPoly.shp', 'w', 'ESRI Shapefile',crs=fiona.crs.from_epsg(4326), schema={'geometry': 'Polygon', 'properties': {}}) as dst:
    for shape in shapes:
        value = shape[1]
        if value > 0:
            feature = {'geometry': shape[0], 'properties': {}}
            dst.write(feature)

# # (Optional) Display the polygons using matplotlib
# with fiona.open(OutputFilePath + 'HeatPoly.shp', 'r') as src:
#     for i, layer in enumerate(src):
#         if i == 0:
#             x, y = [], []
#             for poly in layer['geometry']['coordinates']:
#                 x += [pt[0] for pt in poly]
#                 y += [pt[1] for pt in poly]
#             plt.plot(x, y, 'k.')
#             plt.axis('equal')
#             plt.show()


In [48]:
import geopandas as gpd

# Define the source and target CRSs
src_crs = 'EPSG:4326'  # EPSG:4326 is the default CRS for the input shapefile
target_crs = 'EPSG:26910' # New projection NAD 1983 UTM Zone 10N

# Read the input shapefile into a GeoDataFrame
gdf = gpd.read_file(OutputFilePath + 'HeatPoly.shp', crs=src_crs)

# Reproject the GeoDataFrame to the target CRS
gdf = gdf.to_crs(target_crs)

# Write the reprojected GeoDataFrame to a new shapefile
gdf.to_file(OutputFilePath + 'HeatPoly_reprojected.shp', driver='ESRI Shapefile')


In [49]:
import geopandas as shp
from shapely.ops import unary_union
from shapely.geometry import MultiPolygon

# Load your polygons into a GeoDataFrame
polygons = shp.read_file(work_dir + 'HeatPoly_reprojected.shp')

# Set the distance within which you want to aggregate polygons
distance = 40  # in meters

# Create a buffer around each polygon using the distance
buffered = polygons.geometry.buffer(distance)

# Group buffered polygons that intersect with each other
groups = buffered.unary_union

# Convert the grouped polygons back to a GeoDataFrame
# Convert the grouped polygons back to a GeoDataFrame
if isinstance(groups, MultiPolygon):
    polygons_list = [polygon for polygon in groups.geoms]
else:
    polygons_list = [groups]

grouped_polygons = gpd.GeoDataFrame(
    {'geometry': polygons_list},
    crs=polygons.crs
)

# Save the aggregated polygons to a shapefile
grouped_polygons.to_file(OutputFilePath + 'AggrigateEx1.shp')


In [50]:
# Read in the feature class
fc = gpd.read_file(OutputFilePath + 'AggrigateEx1.shp')

# Define a minimum area threshold for interior polygons (in square units of the CRS)
threshold = 500

# Iterate over each polygon in the feature class
for index, row in fc.iterrows():
    # Get the exterior and interior polygons
    exterior = row.geometry.exterior
    interiors = row.geometry.interiors
    
    # Filter out small interior polygons
    interiors_filtered = [interior for interior in interiors if interior.area > threshold]
    
    # Create a new polygon with the filtered interiors
    filtered_polygon = type(row.geometry)(exterior, interiors_filtered)
    
    # Replace the original geometry with the filtered geometry
    fc.loc[index, 'geometry'] = filtered_polygon
    
# Save the updated feature class
fc.to_file(OutputFilePath + 'Buff.shp')

In [51]:
# Load your polygons into a GeoDataFrame
polygons = gpd.read_file(work_dir + 'Buff.shp')

# Set the distance within which you want to aggregate polygons
distance = -43  # in meters

# Create a buffer around each polygon using the distance
buffered = polygons.geometry.buffer(distance)

# Group buffered polygons that intersect with each other
groups = buffered.unary_union

# Convert the grouped polygons back to a GeoDataFrame
# Convert the grouped polygons back to a GeoDataFrame
if isinstance(groups, MultiPolygon):
    polygons_list = [polygon for polygon in groups.geoms]
else:
    polygons_list = [groups]

grouped_polygons = gpd.GeoDataFrame(
    {'geometry': polygons_list},
    crs=polygons.crs
)

# Save the aggregated polygons to a shapefile
grouped_polygons.to_file(OutputFilePath + 'NegBuff.shp')



In [52]:
# Read in the polygon shapefile
polygons = gpd.read_file(OutputFilePath + 'NegBuff.shp')

# Dissolve the polygons based on a column called 'column_name'
dissolved_polygons = polygons.dissolve(by='FID')

# Save the dissolved polygons to a new shapefile
dissolved_polygons.to_file(OutputFilePath + 'Dissolve.shp')

In [53]:
# # Read in the feature class
# fc = gpd.read_file(OutputFilePath + 'Union.shp')

# # Define a minimum area threshold for interior polygons (in square units of the CRS)
# threshold = 1

# # Iterate over each polygon in the feature class
# for index, row in fc.iterrows():
#     # Get the exterior and interior polygons
#     exterior = row.geometry.exterior
#     interiors = row.geometry.interiors
    
#     # Filter out small interior polygons
#     interiors_filtered = [interior for interior in interiors if interior.area > threshold]
    
#     # Create a new polygon with the filtered interiors
#     filtered_polygon = type(row.geometry)(exterior, interiors_filtered)
    
#     # Replace the original geometry with the filtered geometry
#     fc.loc[index, 'geometry'] = filtered_polygon
    
# # Save the updated feature class
# fc.to_file(OutputFilePath + 'Union2.shp')